## Making Lane Detector

In [1]:
import cv2 as cv
import numpy as np

In [5]:
class PrepareImage():
    '''ATTRIBUTES:
    gauss_size : kernel size for the gaussian blur 
                type-> tuple of size 2 with odd and equal 
                       entries > 1
    gauss_deviation : x and y axis standard deviations for 
               gaussian blur 
               type -> list-like of size = 2
    auto_canny : If auto canny is True use median of blurred 
                image to calculate thresholds 
                type-> boolean
    canny_low : the lower threshold of the canny filter 
                type -> int 
    canny_high : the higher threshold of the canny filter 
                type -> int 
    segment_x : the width of segment peak( the triangular 
               segment head). Given as the fraction of the width 
               of the image
               type -> float in (0,1) 0 and 1 exclusive
    segment_y : the height segment peak
                Given as the fraction of the height from the 
                top 
                type -> float in (0,1) 0 and 1 exclusive
                
    METHODS:
    ...
    '''
    def __init__(self,
                gauss_size = None,
                gauss_deviation = None,
                auto_canny = False,
                canny_low = 50,
                canny_high = 175,
                segment_x = 0.5,
                segment_y = 0.5):
        
        # setting gaussian kernel parameters.
        if(gauss_size is not None):
            if(len(gauss_size) != 2):
                raise Exception("Wrong size for the Gaussian Kernel")
            elif(type(gauss_size) is not tuple):
                raise Exception("Kernel type should be a tuple")
            elif(gauss_size[0]%2 == 0 or gauss_size[1]%2 == 0):
                raise Exception("Even entries found in Gaussian Kernel")    
        self.gauss_kernel = gauss_size
            
        if(gauss_deviation is not None):
            if(len(gauss_deviation)!=2):
                raise Exception("Wrong length of gauss deviation")
            else:
                self.gauss_deviation = gauss_deviation
            
        if(type(auto_canny) is not bool):
            raise TypeError("Incorrect Type mentioned for auto canny")
            
        # setting canny parameters
        if(auto_canny is False):
            self.auto_canny = False
            if(type(canny_high) is int and type(canny_low) is int):
                self.canny_low = canny_low 
                self.canny_high = canny_high 
            else:
                raise TypeError("Incorrect type specified for canny thresholds")
        else:
            self.auto_canny = True
            
        # setting segment parameters
        if segment_x >=1 or segment_x<=0:
            raise Exception("Fraction specified is out of range (0,1)")
        else:
            self.segment_x = segment_x
        if segment_y >=1 or segment_y<=0:
            raise Exception("Fraction specified is out of range (0,1)")
        else:
            self.segment_y = segment_y 
    def do_canny(self,frame):
        '''PARAMETERS: the frame of the image on which we want to apply the 
                      canny filter 
          RETURNS : a canny filtered frame '''
        # gray the image 
        gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY) 
        # apply blur 
        if(self.gauss_kernel is None):
            self.gauss_kernel = (9,9) # using a default kernel size 
        if(self.gauss_deviation is None):
            self.gauss_deviation = [3,3]
        
        blur = cv.GaussianBlur(gray, self.gauss_kernel, self.gauss_deviation[0], self.gauss_deviation[1])
        
        #apply canny filter 
        if self.auto_canny is False:
            canny = cv.Canny(blur,self.canny_low,self.canny_high)
        else:
            # Auto canny trumps specified parameters 
            v = np.median(blur)
            sigma = 0.33
            lower = int(max(0, (1.0 - sigma) * v))
            upper = int(min(255, (1.0 + sigma) * v))
            canny = cv.Canny(blur,lower,upper)
        
        return canny 
    
    def segment_image(self,frame):
        '''PARAMETERS: the frame of the image on which we want to apply the 
                      segementation filter 
        RETURNS : a segmented canny filtered frame '''
        height = frame.shape[0]
        width = frame.shape[1]
        shift = int(0.08 * width)
        points = np.array([
            [(0,height),(width,height),(int(width*self.segment_x)+shift,int(height*self.segment_y)),
             (int(width*self.segment_x)-shift,int(height*self.segment_y))]
        ])
        # create an image with zero intensity with same dimensions as frame.
        mask = np.zeros_like(frame)
        
        cv.fillPoly(mask,points,255) # filling the frame's triangle with white pixels
        # do a bitwise and on the canny filtered black and white image and the 
        # segment you just created to get a triangular area for lane detection 
        segment = cv.bitwise_and(frame, mask)
        
        # boundary lines...
        cv.line(segment,(0,height),(int(width*self.segment_x)-shift,int(height*self.segment_y)),(250,0,0),1)
        cv.line(segment,(width,height),(int(width*self.segment_x)+shift,int(height*self.segment_y)),(250,0,0),1)
        cv.line(segment,(int(width*self.segment_x)+shift,int(height*self.segment_y)),
             (int(width*self.segment_x)-shift,int(height*self.segment_y)),(250,0,0),1)
        
        return segment 
    
    def get_poly_maskpoints(self,frame):
        height = frame.shape[0]
        width = frame.shape[1]
        shift = int(0.08 * width)
        points = np.array([
                [(0,height),(width,height),(int(width*self.segment_x)+shift,int(height*self.segment_y)),
                 (int(width*self.segment_x)-shift,int(height*self.segment_y))]
            ])
        left = (points[0][0],points[0][3])
        right = (points[0][1],points[0][2])
        return (left,right)
    
    def get_binary_image(self,frame):
        can = self.do_canny(frame)
#         cv.imshow(can)
        seg = self.segment_image(can)
        return seg
    

In [42]:
class Curve():
    '''PARAMETERS: sample_points -> int: the number of samples of lines 
                                    which are used to approximate the 
                                    curve 
                    color -> 3-tuple: (r,g,b) the color of the curve 
                    fill -> Boolean: whether to fill the curve or not 
                    window_size -> float (0,1): how wide you want the 
                                    window to be 
    '''
    def __init__(self,
                window_size = 0.2):
        
        self.left_coords = []
        self.right_coords = []
        if(window_size >=1 or window_size<=0):
            raise Exception("Invalid window size given") 
        self.window_size = window_size
        
        
    def find_non_zero(self,frame):
        '''
        Finds all the non zero points inside the Trapezium boundary
        Faster than cv.FindNonZero...
        PARAMETERS: frame : binary image for which we want non zero x,y coords
        RETURNS: a list of 2-tuples of the non zero points , sorted by x 
        '''
        # this code is written manually to speed up computational cost
        half = int(frame.shape[0]/2) # height / 2
        row = frame.shape[0] - 2
        left, right = 0,frame.shape[1] 
        width = frame.shape[1]
        points =[]
        while row > half and left + int(0.08*width)< right - int(0.08*width):
            for i in range(left,right):
                if(frame[row][i] != 0):
                    points.append((row,i))
            left+=2
            right-=2
            row-=2
        return sorted(points)
    
    def isInside(self,point,left_lane,right_lane):
        '''Function to check whether the point given 
        lies inside the parallelogram or not 
        Cross product of all pairwise triangles must add 
        up to the area of parallelogram. 
        PARAMETERS: point ,> a 2,tuple which contains the 
                            co,ordinates of the point 
                    left_lane ,>2,tuple which consists of  the 
                            left side of the parallelogram
                    right_lane ,>2 ,tuple which consists of the 
                            right side of the parallelogram 
        RETURNS : Boolean specifying whether the point lies 
                  inside or not '''
        bottom_left = left_lane[0]
        bottom_right = right_lane[0]
        top_left = left_lane[1]
        top_right = right_lane[1]
        
        # start from top 
        A1B = np.subtract(top_left,point)
        A1C = np.subtract(top_right,point )
        a1 = 0.5 * abs(np.cross(A1B,A1C))
        
        # right 
        A2B = np.subtract(top_right,point )
        A2C = np.subtract(bottom_right,point)
        a2 = 0.5 * abs(np.cross(A2B,A2C))
        
        #bottom 
        A3B = np.subtract(bottom_right,point) 
        A3C = np.subtract(bottom_left,point)
        a3 = 0.5 * abs(np.cross(A3B,A3C))
        
        #left 
        A4B = np.subtract(bottom_left,point) 
        A4C = np.subtract(top_left,point) 
        a4 = 0.5 * abs(np.cross(A4B,A4C))
        
        AB = np.subtract(top_left,bottom_left)
        AC = np.subtract(bottom_right,bottom_left) 
        A = abs(np.cross(AB,AC))
        
        if(a1+a2+a3+a4 == A):
            return True
        else:
            return False
        
    def get_left_curve(self,frame,left_lane):
        '''Method to calculate the left fit curve for the 
        given frame 
        PARAMETERS: frame : the image frame 
                    left_lane : the boundary left lane of segmented image
        RETURNS: None : if no curve detected 
                 3-tuple: if curve detected (A,B,C) : coefficients of 
                    x^2, x and the constant factor in Ax^2 + Bx + C'''
        if(left_lane is None):
            raise Exception ("No left lane given")
            return 
        
        width = frame.shape[1]
        shift = self.window_size * width
        # start drawing windows and fitting curves 
        xy = self.find_non_zero(np.array(frame))
        left = left_lane  #-> like this - '/'
        # get the right lane -> '/'
        right = ((left[0][0] + shift,left[0][1]),(left[1][0]+shift,left[1][1]))
        
        while right[0][0] < int(0.5 * width):
            # get all the points that are non zero inside the parallelogram 
            # and get there x-y coords 
            X, Y =[],[]
            for k in xy:
                if(self.isInside(k,left,right) == True): 
                    # to - do
                    X.append(k[0])
                    Y.append(k[1])
            '''Only calculate the parabola if you actually 
            have more than 100 points to fit the curve to.
            This is because the points may just be noise if they are 
            very less'''
            if(len(X) <= 150):
                # shift window
                left = right 
                right = ((left[0][0] + shift,left[0][1]),(left[1][0]+shift,left[1][1]))
                continue 
            # polyfit returns a 3 tuple with the 
            # x^2 coefficient as 1st element, x as 2nd and constant as 3rd
            parabola = np.polyfit(X,Y,2)
            # save polynomial coords 
            left = right 
            right = ((left[0][0] + shift,left[0][1]),(left[1][0]+shift,left[1][1]))
            
            self.left_coords.append(parabola)
            
        A,B,C = [],[],[]
        for k in self.left_coords:
            A.append(k[0])
            B.append(k[1])
            C.append(k[2])
        #average out 
        try:
            A = sum(A)/len(A)
            B = sum(B)/len(B)
            C = sum(C)/len(C)
        except:
            return None
        return (A,B,C)
    
    def get_right_curve(self,frame,right_lane):
        '''Method to calculate the left fit curve for the 
        given frame 
        PARAMETERS: frame : the image frame 
                    left_lane : the boundary left lane of segmented image
        RETURNS: None : if no curve detected 
                 3-tuple: if curve detected (A,B,C) - coefficients of 
                    x^2, x and the constant factor in Ax^2 + Bx + C'''
        if(right_lane is None):
            raise Exception ("No right lane given")
            return 
        width = frame.shape[1]
        shift = self.window_size * width
        # get all non-zero x,y coordinates sorted by x 
        xy = self.find_non_zero(np.array(frame))

        # start drawing windows and fitting curves 
        right = right_lane  #-> like this - '\'
        # get the left lane - '\'
        left = ((right[0][0] - shift,right[0][1]),(right[1][0]-shift,right[1][1]))
        while left[0][0] > int(0.5 * width):
            
            # get all the non -zero points that are inside the parallelogram 
            # and get there x-y coords 
            X, Y =[],[]
            for k in xy:
                if(self.isInside(k,left,right) == True): 
                    # to - do
                    X.append(k[0])
                    Y.append(k[1])
            '''Only calculate the parabola if you actually 
            have like more than 50 points to fit the curve to.
            This is because the points may just be noise if they are 
            very less'''
            if(len(X) <= 150):
                # shift window 
                right = left 
                left = ((right[0][0] - shift,right[0][1]),(right[1][0]-shift,right[1][1]))
                continue 
                
            # polyfit returns a 3 tuple with the 
            # x^2 coefficient as 1st element, x as 2nd and constant as 3rd
            parabola = np.polyfit(X,Y,2)
            # save polynomial coords 
            self.right_coords.append(parabola)
            
            
        A,B,C = [],[],[]
        for k in self.right_coords:
            A.append(k[0])
            B.append(k[1])
            C.append(k[2])
        
        #average out 
        try:
            A = sum(A)/len(A)
            B = sum(B)/len(B)
            C = sum(C)/len(C)
        # tuple with right lane parameters
        except:
            return None    
        return (A,B,C)
        

In [43]:
class Plotter():
    '''A classs to actually plot the detected curves on the frame 
    of the video 
    PARAMETERS:  color - (3-tuple) specifying the values of the (R,G,B) channels
                 step - (int) It defines the pixel step required in plotting the 
                        curve. Specifies how detailed potting is required.
                        Smaller step means smoother curve 
                 '''
    def __init__(self,color = (0,255,0),
                step = 2,
                thickness = 2):
        self.color = color
        
        if(type(thickness) is not int or thickness < 1):
            raise Exception("Invalid line thickness given for plotting")
        self.thickness = thickness
        
        if(type(step) is not int or step < 1):
            raise Exception("Invalid step size given for plotting")
        self.step = step 
        
    def get_y(self,x,parabola):
        '''Returns the value of ax^2 + bx + c'''
        A = parabola[0]
        B = parabola[1]
        C = parabola[2]
        return int(A*(x**2) + B*x + C)
    
    def plot_buffer(self,frame,limits,orientation):
        '''Plot the buffer lane on the curve if no curve has 
           been detected.
           RETURNS: frame with the buffer lane drawn '''
        height = frame.shape[0]
        end_height = int(0.5 * height) # end point of buffer
        if orientation == 'L':
            # draw left buffer 
            # left limit starts from far left point
            left = limits[0]
            right = limits[1]
            #define points
            p1 = (left,height-1)
            p2 = (right,end_height)
            cv.line(frame,p1,p2,self.color,self.thickness)
        
        else:
            # draw right buffer 
            # right limit starts from far right point
            right = limits[0]
            left = limits[1]
            #define points 
            
            p1 = (right,height-1)
            p2 = (left,end_height)
            cv.line(frame,p1,p2,self.color,self.thickness)
        
        return frame 
            
    def plot_curve_left(self,frame,limits,parabola,buffer_needed):
        '''Method to plot the left parabola on the given 
           frame. If no parabola detected, buffer lane is 
           plotted on the image '''
        height = frame.shape[0]
        if buffer_needed == True:
            frame = self.plot_buffer(frame,limits,'L')
            return frame
        else:
            x_start = limits[0] # left end
            x_end = limits[1] # right end 
            
            # get the first point on your image 
            left = x_start
            right = left + self.step 
            while right < x_end :
                # now generate co-ordinates of the points 
                # according to the parabola 
                y1 = self.get_y(left,parabola)
                y2 = self.get_y(right,parabola)
                
                # need to check if coordinates are actually inside the image 
                if y1 >= height or y2>= height:
                    left = right 
                    right = left + self.step 
                    continue
                # if okay 
                p1 = (left,y1)
                p2 = (right,y2)
                
                #plot this line on the frame
                cv.line(frame,p1,p2,self.color,self.thickness)
                left = right 
                right = left + self.step 
        
            return frame
        
    def plot_curve_right(self,frame,limits,parabola,buffer_needed):
        '''Method to plot the right parabola on the given 
           frame. If no parabola detected, buffer lane is 
           plotted on the image '''
        height = frame.shape[0]
        if buffer_needed == True:
            frame = self.plot_buffer(frame,limits,'R')
            return frame
        else:
            x_start = limits[0] # right end
            x_end = limits[1] # left end
            
            # get the first point on your image 
            right = x_start
            left = right - self.step 
            while left > x_end :
                # now generate co-ordinates of the points 
                # according to the parabola 
                y1 = self.get_y(left,parabola)
                y2 = self.get_y(right,parabola)
                # need to check that y co-ordinates lie in the image first 
                if y1 >= height or y2>= height:
                    right = left 
                    left = right - self.step 
                    continue 
                # if okay, 
                p1 = (left,y1)
                p2 = (right,y2)
                
                #plot this line on the frame
                cv.line(frame,p1,p2,self.color,self.thickness)
                right = left 
                left = right - self.step
                
            return frame
        
        

In [44]:
def rescale_frame(frame,percent=75):
        width = int(frame.shape[1] * percent / 100)
        height = int(frame.shape[0] * percent / 100)
        dim = (width,height)
        return cv.resize(frame,dim,interpolation = cv.INTER_AREA)

In [45]:
prep = PrepareImage((11,11),(3,3),auto_canny=True, segment_y = 0.5)
CurveMake = Curve()
plotter = Plotter((0,255,0),6,2)

In [46]:
# working fine...
cap = cv.VideoCapture("E:\InnerveHackathon\pathvalild.mp4")
while (cap.isOpened()):
    # ret = a boolean return value from getting the frame,
    # frame = the current frame being projected in video
    ret, frame = cap.read()
    try:
        frame = rescale_frame(frame,percent = 57)
    except:
        break
    width , height = frame.shape[1], frame.shape[0]
    cv.imshow("Original",frame)
    frame = prep.get_binary_image(frame)
    points = prep.get_poly_maskpoints(frame)

    left_lane = points[0] # two tuple 
    right_lane = points[1] # two tuple
    
    # pass these co-ordinates and the frame in a new class which 
    # actually gets the polynomial fitted
    left_coords = CurveMake.get_left_curve(frame,left_lane)
    right_coords = CurveMake.get_right_curve(frame,right_lane)
    
    # to incorporate the fact that the curve has not been detected
    # we keep 2 boolean values
    buffer_left = False 
    buffer_right = False

    #  now plot this curve on your image if you get it, else plot the base curve 
    if(left_coords is None):
        buffer_left = True
    if(right_coords is None):
        buffer_right = True 
    
    #define limits -> only in this range I shall plot my curve
    limit_left_x = (int(0.05*width),int(0.40*width)) # left to right
    limit_right_x = (int(0.95*width),int(0.55*width)) # right to left
    
    # plot on image
    frame = plotter.plot_curve_left(frame,limit_left_x,left_coords,buffer_left)
    frame = plotter.plot_curve_right(frame,limit_right_x,right_coords,buffer_right)
    
    #show image
    
# cv.line(images[1][0],(images[1][0].shape[1]//2,0),(images[1][0].shape[1]//2,images[1][0].shape[0]),(200,200,0),5)
    cv.imshow("Final",frame)
    if cv.waitKey(7) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

## So, we will show atleast a basic navigation information if no lane is actually detected
- Made the buffer lanes for the output for lanes when there actually isn't any lane detected throgh Hough Transformations


In [53]:
frame = cv.imread(r"E:\InnerveHackathon\openCV Lanes\Detection Stages and  Examples\cannyOrig.jpg")
# frame = rescale_frame(frame)
frame = prep.get_binary_image(frame)

# cv.imshow("frame",frame)
# xy = []
xy = cv.findNonZero(np.array(frame))
points =[]
print("Width :",frame.shape[1])
for k in xy:
    for j in k:
        points.append((j[0],j[1]))
points = sorted(points)
# print(points)
# cv.waitKey(4000)
# cv.destroyAllWindows()

Width : 1280


In [4]:
abs(np.cross([1,1],[1,0]))

1

In [30]:
left = ((1,0),(1,3))
right = ((3,0),(3,3))
point = (3,3)
print(isInside(point,left,right))

True
